# Investigating Football Referee Data Across the Big 5 Leagues 

## Importing Data

In [1]:
import pandas as pd
import numpy as np
import seaborn as sns
from matplotlib import pyplot as plt

In [2]:
#open our csv file and assign it to a variable
ref_data = pd.read_csv("top 5 leagues ref data.csv")
#preview our dataset (Data correct as of 29/10/24)
ref_data.head()

,name,league,born,country,year,matches,y,yr,r
0,Michael Oliver,premier league,20/02/1985,England,24/25,8.0,43.0,1,-
1,Anthony Taylor,premier league,20/10/1978,England,24/25,8.0,45.0,-,1
2,Chris Kavanagh,premier league,04/09/1985,England,24/25,7.0,48.0,2,1
3,Sam Barrott,premier league,NaN,England,24/25,6.0,39.0,-,-
4,Robert Jones,premier league,04/04/1987,England,24/25,6.0,30.0,1,3


## Inspecting Data

In [3]:
#inspect the data types in our dataset 
ref_data.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 384 entries, 0 to 383
Data columns (total 9 columns):
 #   Column   Non-Null Count  Dtype  
---  ------   --------------  -----  
 0   name     381 non-null    object 
 1   league   381 non-null    object 
 2   born     307 non-null    object 
 3   country  381 non-null    object 
 4   year     381 non-null    object 
 5   matches  381 non-null    float64
 6   y        381 non-null    float64
 7   yr       381 non-null    object 
 8   r        381 non-null    object 
dtypes: float64(2), object(7)
memory usage: 27.1+ KB


In [4]:
#inspect the key statistical values for our float data to help better understand our dataset
ref_data.describe()

,matches,y
count,381.000000,381.000000
mean,10.559055,44.212598
std,7.123704,30.586107
min,1.000000,2.000000
25%,4.000000,17.000000
50%,10.000000,42.000000
75%,16.000000,66.000000
max,30.000000,130.000000


## Cleaning Data

In [5]:
#Drop the "born" & "country" columns to reduce clutter
ref_data = ref_data.drop(labels=["born", "country"], axis=1)

In [6]:
#Remove the whitespace from the column names
for column in ref_data.columns:
    column = column.strip() 

#Replace the column names so they are easier to understand
column_mapper = {"name":"Referee",
                 "league":"League",
                 "year":"Season",
                 "matches":"Number of Matches",
                 "y":"Yellows",
                 "yr":"Double Yellows",
                 "r":"Reds"}
ref_data = ref_data.rename(mapper=column_mapper, axis=1)

In [7]:
#Title the data in the "League" column to make it more presentable
ref_data["League"] = ref_data["League"].str.title()

In [8]:
#Replace the "-" in the "Yellows", "Double Yellows" and "Reds", with "0"s
ref_data["Double Yellows"] = ref_data["Double Yellows"].str.replace(pat="-", repl="0", regex=False)
ref_data["Reds"] = ref_data["Reds"].str.replace(pat="-", repl="0", regex=False)

In [9]:
#Find and drop any rows with NaN values them to ensure we can perform mathematical operations on our data
ref_data = ref_data.dropna(axis=0)

In [10]:
#Investigate the index count to see how many rows have been dropped
ref_data.info()

<class 'pandas.core.frame.DataFrame'>
Index: 381 entries, 0 to 383
Data columns (total 7 columns):
 #   Column             Non-Null Count  Dtype  
---  ------             --------------  -----  
 0   Referee            381 non-null    object 
 1   League             381 non-null    object 
 2   Season             381 non-null    object 
 3   Number of Matches  381 non-null    float64
 4   Yellows            381 non-null    float64
 5   Double Yellows     381 non-null    object 
 6   Reds               381 non-null    object 
dtypes: float64(2), object(5)
memory usage: 23.8+ KB


We can confirm **3 rows have been dropped** (= 384 - 381)

In [11]:
#Convert all of our columns containing floats to integers
for column in ref_data.columns:
    if ref_data[column].dtype == "float64":
        ref_data[column] = ref_data[column].astype("int64")
        
#Convert the "Double Yellows" and "Reds" object columns to integers
ref_data["Double Yellows"] = ref_data["Double Yellows"].astype("int64")
ref_data["Reds"] = ref_data["Reds"].astype("int64")

#Verify we have the correct column types
ref_data.info()

<class 'pandas.core.frame.DataFrame'>
Index: 381 entries, 0 to 383
Data columns (total 7 columns):
 #   Column             Non-Null Count  Dtype 
---  ------             --------------  ----- 
 0   Referee            381 non-null    object
 1   League             381 non-null    object
 2   Season             381 non-null    object
 3   Number of Matches  381 non-null    int64 
 4   Yellows            381 non-null    int64 
 5   Double Yellows     381 non-null    int64 
 6   Reds               381 non-null    int64 
dtypes: int64(4), object(3)
memory usage: 23.8+ KB


In [12]:
#Refine the data to ensure that each referee has taken charge of at least 3 games in a season, limiting the effect variance has on our results
ref_data = ref_data[ref_data["Number of Matches"] >= 3]

## Preparing Data

In [13]:
#Create a function that calcultes "Total Booking Points", which is equal to: ("Yellows" * 10) + ("Double Yellows" * 35) + ("Reds" * 25)
def total_bp(df, yellows = "Yellows", double_yellows = "Double Yellows", reds = "Reds"):
    df["Total Booking Points"] = (df[yellows] * 10) + (df[double_yellows] * 35) + (df[reds] * 25)
    return df

#Create another function that calculates "Booking Points per Match", equal to: "Total Booking Points" / "Number of Matches", rounded to 1 dp
def bp_match(df, booking_points = "Total Booking Points", matches = "Number of Matches"):
    df["Booking Points per Match"] = (df[booking_points] / df[matches]).round(1)
    return df

#Apply the custom functions to our dataframe to create 2 new columns
total_bp(ref_data)
bp_match(ref_data)


,Referee,League,Season,Number of Matches,Yellows,Double Yellows,Reds,Total Booking Points,Booking Points per Match
0,Michael Oliver,Premier League,24/25,8,43,1,0,465,58.1
1,Anthony Taylor,Premier League,24/25,8,45,0,1,475,59.4
2,Chris Kavanagh,Premier League,24/25,7,48,2,1,575,82.1
3,Sam Barrott,Premier League,24/25,6,39,0,0,390,65.0
4,Robert Jones,Premier League,24/25,6,30,1,3,410,68.3
...,...,...,...,...,...,...,...,...,...
377,Martin Petersen,Bundesliga,22/23,10,48,0,0,480,48.0
378,Tobias Welz,Bundesliga,22/23,10,51,0,4,610,61.0
379,Patrick Ittrich,Bundesliga,22/23,9,32,0,0,320,35.6
380,Tobias Reichel,Bundesliga,22/23,9,32,0,0,320,35.6


In [14]:
#Identify all the leagues present in our dataframe
print(ref_data["League"].unique())

['Premier League' 'La Liga' 'Serie A' 'Ligue 1' 'Bundesliga']


In [15]:
#Group the referees by League and Season
refs_prem2425 = ref_data[(ref_data["League"] == "Premier League") & (ref_data["Season"] == "24/25")].copy()
refs_prem2324 = ref_data[(ref_data["League"] == "Premier League") & (ref_data["Season"] == "23/24")].copy()
refs_prem2223 = ref_data[(ref_data["League"] == "Premier League") & (ref_data["Season"] == "22/23")].copy()

refs_laliga2425 = ref_data[(ref_data["League"] == "La Liga") & (ref_data["Season"] == "24/25")].copy()
refs_laliga2324 = ref_data[(ref_data["League"] == "La Liga") & (ref_data["Season"] == "23/24")].copy()
refs_laliga2223 = ref_data[(ref_data["League"] == "La Liga") & (ref_data["Season"] == "22/23")].copy()

refs_seriea2425 = ref_data[(ref_data["League"] == "Serie A") & (ref_data["Season"] == "24/25")].copy()
refs_seriea2324 = ref_data[(ref_data["League"] == "Serie A") & (ref_data["Season"] == "23/24")].copy()
refs_seriea2223= ref_data[(ref_data["League"] == "Serie A") & (ref_data["Season"] == "22/23")].copy()

refs_ligue12425 = ref_data[(ref_data["League"] == "Ligue 1") & (ref_data["Season"] == "24/25")].copy()
refs_ligue12324 = ref_data[(ref_data["League"] == "Ligue 1") & (ref_data["Season"] == "23/24")].copy()
refs_ligue12223 = ref_data[(ref_data["League"] == "Ligue 1") & (ref_data["Season"] == "22/23")].copy()

refs_bundes2425 = ref_data[(ref_data["League"] == "Bundesliga") & (ref_data["Season"] == "24/25")].copy()
refs_bundes2324 = ref_data[(ref_data["League"] == "Bundesliga") & (ref_data["Season"] == "23/24")].copy()
refs_bundes2223 = ref_data[(ref_data["League"] == "Bundesliga") & (ref_data["Season"] == "22/23")].copy()

In [16]:
#Create an array containing our dataframes
dfs = [refs_prem2425, refs_prem2324, refs_prem2223,
       refs_laliga2425, refs_laliga2324, refs_laliga2223,
       refs_seriea2425, refs_seriea2324, refs_seriea2223,
       refs_ligue12425, refs_ligue12324, refs_ligue12223,
       refs_bundes2425, refs_bundes2324, refs_bundes2223]

In [17]:
#Calculate the strictness of the referees in each league season by dividing their "Booking Points per Match" by the league average for that season
for df in dfs:
    for referee in df["Referee"]:
        df["Strictness%"] = ((df["Booking Points per Match"]/(df["Total Booking Points"].sum()/df["Number of Matches"].sum()))*100).round(1)

#Concatenate our modified data frames, replace the orignal and reset the index
ref_data = pd.concat(dfs).reset_index()
ref_data.head()

,index,Referee,League,Season,Number of Matches,Yellows,Double Yellows,Reds,Total Booking Points,Booking Points per Match,Strictness%
0,0,Michael Oliver,Premier League,24/25,8,43,1,0,465,58.1,96.7
1,1,Anthony Taylor,Premier League,24/25,8,45,0,1,475,59.4,98.9
2,2,Chris Kavanagh,Premier League,24/25,7,48,2,1,575,82.1,136.7
3,3,Sam Barrott,Premier League,24/25,6,39,0,0,390,65.0,108.2
4,4,Robert Jones,Premier League,24/25,6,30,1,3,410,68.3,113.7


In [28]:
#Calculate the mean booking points for each season in each league and store these values in an array called "averages"
averages = []
def avgBP(df):
    return df["Total Booking Points"].sum() / df["Number of Matches"].sum()

for df in dfs:
    averages.append(avgBP(df).round(2))

#Create an array for each league season
league_season = ["Prem2425",
                 "Prem2324",
                 "Prem2223",
                 "LaLiga2425",
                 "LaLiga2324",
                 "LaLiga2223",
                 "SerieA2425",
                 "SerieA2324",
                 "SerieA2223",
                 "Ligue12425",
                 "Ligue12324",
                 "Ligue12223",
                 "Bundes2425",
                 "Bundes2324",
                 "Bundes2223"]

#Create a dictionary that combines these arrays
league_season_averages = dict(zip(league_season, averages))

## Analysing Data

In [19]:
#Sort the "league_season_averages" dictionary so we can compare leagues
league_season_averages_sorted = sorted(league_season_averages, key=league_season_averages.get, reverse=True)
for x in league_season_averages_sorted:
    print(x, league_season_averages[x])

Prem2425 60.06
LaLiga2223 58.63
LaLiga2425 54.27
LaLiga2324 53.36
Ligue12425 51.05
SerieA2223 50.84
Bundes2425 50.0
SerieA2425 48.97
SerieA2324 48.49
Prem2324 47.46
Bundes2324 47.0
Bundes2223 45.81
Ligue12324 44.44
Ligue12223 39.43
Prem2223 38.64


It appears that the **Premier League** has both the **highest and lowest** league average of the last 3 seasons. The highest coming from the current 24/25 season and the lowest coming from the 22/23 season. One possible explanation for this big increase could be the recent crackdown from the Premier League on dissent and time wasting, something that could be further investigated using data on the causes of cards, but that is beyond this dataset.

In [20]:
#Use a pivot_table to observe the variance across the leagues in each season with a 3-game threshold
league_season_var_3 = pd.pivot_table(ref_data,
                                   index="Season",
                                   columns="League",
                                   values="Strictness%",
                                   aggfunc="var").sort_values(by="Season", ascending=False).round(0)

league_season_var_3

League,Bundesliga,La Liga,Ligue 1,Premier League,Serie A
Season,,,,,
24/25,1359.0,1054.0,405.0,319.0,648.0
23/24,246.0,231.0,113.0,279.0,386.0
22/23,476.0,272.0,219.0,239.0,451.0


Upon an initial glance, the Bundesliga 24/25 and La Liga 24/25 seasons exhibit extremely high variance - most likely a consequence of smaller sample sizes when compared to previous seasons. In fact, the unfinished 24/25 season displays the highest season varaince for every single one of the 5 leagues, further reinforcing this point. The expectation is that, as these sample sizes increase, this variance will decrease.

### Investigating the Bundesliga's high variance this season

In [21]:
#Display the Bundesliga 24/25 table to help us find any immediately obvious explanations
refs_bundes2425.sort_values(by="Number of Matches", ascending=False)

,Referee,League,Season,Number of Matches,Yellows,Double Yellows,Reds,Total Booking Points,Booking Points per Match,Strictness%
311,Florian Badstübner,Bundesliga,24/25,4,16,0,0,160,40.0,80.0
312,Benjamin Brand,Bundesliga,24/25,4,12,0,2,170,42.5,85.0
313,Robert Hartmann,Bundesliga,24/25,4,21,3,0,315,78.8,157.6
314,Sven Jablonski,Bundesliga,24/25,4,21,1,0,245,61.2,122.4
315,Harm Osmers,Bundesliga,24/25,4,19,1,0,225,56.2,112.4
316,Tobias Reichel,Bundesliga,24/25,4,20,0,1,225,56.2,112.4
317,Daniel Siebert,Bundesliga,24/25,4,17,0,0,170,42.5,85.0
318,Felix Zwayer,Bundesliga,24/25,4,18,0,0,180,45.0,90.0
319,Dr. Felix Brych,Bundesliga,24/25,3,9,0,0,90,30.0,60.0
320,Bastian Dankert,Bundesliga,24/25,3,9,0,0,90,30.0,60.0


When inspecting the Bundesliga 24/25 data, we can see that the **most games** any single referee has taken charge of is **4**. This makes their data vulnerable to being skewed heavily by extreme games, such as high-card derbies or low-card, high-supremacy games (where there is a strong favourite and thus the game has a greater chance of being "dead" earlier on). This could be an indication that there are a larger number of extreme fixtures in this leagues compared to other leagues or even that the league is becoming less competitive, but once again that is outside of the scope of this project and more data would be required to investigate this.

In [22]:
#Sort the dataframe by "Strictness%"
ref_data.sort_values(by="Strictness%", ascending=False)

,index,Referee,League,Season,Number of Matches,Yellows,Double Yellows,Reds,Total Booking Points,Booking Points per Match,Strictness%
284,322,Florian Exner,Bundesliga,24/25,3,19,1,3,300,100.0,200.0
79,94,Jesús Gil Manzano,La Liga,24/25,4,34,1,1,400,100.0,184.3
275,313,Robert Hartmann,Bundesliga,24/25,4,21,3,0,315,78.8,157.6
68,83,Juan Pulido Santana,La Liga,24/25,6,39,2,2,510,85.0,156.6
141,156,Antonio Rapuano,Serie A,24/25,3,17,1,1,230,76.7,156.6
...,...,...,...,...,...,...,...,...,...,...,...
122,137,Ermanno Feliciani,Serie A,24/25,4,10,0,0,100,25.0,51.0
292,330,Tobias Welz,Bundesliga,24/25,3,7,0,0,70,23.3,46.6
283,321,Christian Dingert,Bundesliga,24/25,3,7,0,0,70,23.3,46.6
213,242,Maria Caputi,Serie A,22/23,3,7,0,0,70,23.3,45.8


Just from previewing our sorted dataset, we can see that **9 of the 10 extremes are from the current 24/25 season**. This is hardly surprising, as the small nature of the sample sizes means they are more influenced by anomalous data. As the season progresses, and these referees take charge of more games, you would expect their averages to regress towards the league mean. This is visible when we increase the minimum number of matches to 6 games, where we can now see that only 1 referee from the 24/25 season features in the extremes (slight caveat here is that the count of referees who have taken charge of 6 games this season will be reduced compared to previous seasons):

In [23]:
ref_data_6 = ref_data[ref_data["Number of Matches"] >= 6]
ref_data_6.sort_values(by="Strictness%", ascending=False)

,index,Referee,League,Season,Number of Matches,Yellows,Double Yellows,Reds,Total Booking Points,Booking Points per Match,Strictness%
68,83,Juan Pulido Santana,La Liga,24/25,6,39,2,2,510,85.0,156.6
181,210,Davide Massa,Serie A,22/23,14,89,1,4,1025,73.2,144.0
166,190,Giovanni Ayroldi,Serie A,23/24,8,42,3,1,550,68.8,141.9
144,168,Andrea Colombo,Serie A,23/24,16,94,1,5,1100,68.8,141.9
322,364,Harm Osmers,Bundesliga,22/23,15,74,4,3,955,63.7,139.1
...,...,...,...,...,...,...,...,...,...,...,...
177,206,Daniele Orsato,Serie A,22/23,15,45,1,0,485,32.3,63.5
117,132,Javier Alberola Rojas,La Liga,22/23,16,49,0,4,590,36.9,62.9
36,40,Graham Scott,Premier League,23/24,6,17,0,0,170,28.3,59.6
325,367,Tobias Stieler,Bundesliga,22/23,15,36,1,0,395,26.3,57.4


In [24]:
#Use another pivot_table to observe the variance across the leagues in each season with a 6 game threshold
league_season_var_6 = pd.pivot_table(ref_data_6,
                                   index="Season",
                                   columns="League",
                                   values="Strictness%",
                                   aggfunc="var").sort_values(by="Season", ascending=False).round(0)
league_season_var_6

League,Bundesliga,La Liga,Ligue 1,Premier League,Serie A
Season,,,,,
24/25,NaN,578.0,NaN,257.0,NaN
23/24,246.0,231.0,113.0,265.0,419.0
22/23,476.0,272.0,219.0,212.0,394.0


In [25]:
#reprint the 3-game variance threshold for comparison 
league_season_var_3

League,Bundesliga,La Liga,Ligue 1,Premier League,Serie A
Season,,,,,
24/25,1359.0,1054.0,405.0,319.0,648.0
23/24,246.0,231.0,113.0,279.0,386.0
22/23,476.0,272.0,219.0,239.0,451.0


As predicted, **increasing** the threshold on the **number of games** generally leads to a **decrease in the variance**, with the most signficiant drops being across this season.
Some leagues exhibit unchanged variance, which will just mean there were no referees with 3, 4 or 5 games to filter out.

In [26]:
#Create a function that allows us to find league data on any given referee that exists in the dataset 
def ref_finder(referee):
    ref_only = ref_data[ref_data["Referee"] == referee]
    return ref_only

ref_finder("Michael Oliver")

,index,Referee,League,Season,Number of Matches,Yellows,Double Yellows,Reds,Total Booking Points,Booking Points per Match,Strictness%
0,0,Michael Oliver,Premier League,24/25,8,43,1,0,465,58.1,96.7
20,24,Michael Oliver,Premier League,23/24,24,97,2,3,1115,46.5,98.0
40,49,Michael Oliver,Premier League,22/23,30,86,0,1,885,29.5,76.3


### Conclusion

In this project, we imported a dataset containing publically available data on the types of cards shown by football referees across the Big 5 leagues in Europe. We began by previewing the dataset to identify areas of improvement. The dataset was then cleaned and prepared using methods, loops and functions to output a dataframe upon which we could perform our analysis.

Upon analysis we discovered that, surprisingly, the Premier League had both the highest and lowest single-season booking points average of the Big 5 leagues across the last 3 seasons. One potential reason for this new maximum was the Premier League's crackdown on dissent and time-wasting. Whilst this dataset does not have the correct data to answer this question, it is certainly an area that could be investigated further in the future. For example, I believe it would benefit from the use of a ***polynomial regression model*** to identify the driving factor(s) behind this sudden increase and whether there is a stronger weighting towards either dissent or time-wasting. Additionally, it would be of interest to return to this project at the end of the season and see whether this inflated league average is sustainable.

Furthermore, we calculated a variable "Strictness%" which allowed us to compare how a referee performs against the league average in any given season. Upon preliminary investigation, it became evident that 9 of the 10 extremes for referee strictness came from the current 24/25 season. This was unsurprising and likely a result of variance in small samples that are more easily skewed by anomalous data. To investigate this variance further, we employed the use of pivot tables. As expected, we noticed a substantial decrease in the variance once the game-threshold was increased from 3 to 6 matches. Again, another point for further analysis and one to keep an eye on as the season progresses.

Lastly, I believe this analysis would benefit greatly from some visual representation through a data visualisation tool such as Power BI or Tableau.

In [27]:
#Export our full dataframe to allow analysis via PowerBI/Tableau
ref_data.to_csv("Cleaned Referee Data.csv")